In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from owlready2 import *

SQLITE3FILE = os.getcwd() + "/4WD_car_db.sqlite3"
IRI = "http://www.semanticweb.org/fenrir/ontologies/2023/5/4WD_car_specific_ontology"

def get_onto():
    default_world.set_backend(filename=SQLITE3FILE, exclusive=False)
    return default_world.get_ontology(IRI).load()

def close():
    default_world.close()

onto = get_onto()

In [ ]:
df = pd.DataFrame(columns=["timestamp","voltage","charge"])

for car in onto.search(is_a = onto.car):
    for battery in car.hasPart:
        if not isinstance(battery, onto.battery): continue
        for measure in battery.hasMeasure:
            df.loc[len(df)] = pd.Series(dtype=object)
            for prop in measure.get_properties():
                for value in prop[measure]:
                    df.loc[len(df)-1, prop.python_name] = value

In [ ]:
df.head()

In [ ]:
voltage_values = df['voltage'].astype('float64')
timestamps = df['timestamp']

coefficients = np.polyfit(range(len(voltage_values)), voltage_values, 2)
polynomial = np.poly1d(coefficients)

trend_values = polynomial(range(len(timestamps)))

fig, ax = plt.subplots()

ax.plot(df['timestamp'], df['voltage'], label='Voltage')
ax.plot(timestamps, trend_values, label='Trend')


ax.legend()
ax.set_xlabel('Timestamp')
ax.set_ylabel('Voltage')
plt.title('Voltage over Time')

plt.show()

In [ ]:
close()